In [3]:
using ITensors, ITensorMPS, ITensorGaussianMPS, LinearAlgebra, Plots

# Function to construct the hopping Hamiltonian matrix
function hopping_matrix(N::Int, t::Float64)
    H = zeros(Float64, N, N)
    for j in 1:N-1
        H[j, j+1] = -t
        H[j+1, j] = -t
    end
    # Periodic boundary condition
    H[1, N] = -t
    H[N, 1] = -t
    return H
end

# Function to compute the Slater determinant matrix
function compute_slater_matrix(H::Matrix{Float64}, Nf::Int)
    eigvals, eigvecs = eigen(H)
    return eigvecs[:, 1:Nf]  # Take lowest-energy Nf eigenvectors
end

# Function to run DMRG on the interacting SSH model
function run_dmrg_ll(N::Int, t::Float64, V::Float64, nsweeps::Int, maxdim::Vector{Int}, cutoff::Float64, noise::Float64, weight::Float64)
    _maxlinkdim = 1300
    _cutoff = 1e-10
    _eigval_cutoff = 1e-8
    _maxblocksize = 30 
    Nf = N ÷ 2  # Half-filled fermion system

    # Construct free fermion hopping Hamiltonian
    H_free = hopping_matrix(N, t)
    
    # Compute Slater determinant matrix
    Φ = compute_slater_matrix(H_free, Nf)
    
    # Define fermionic sites
    sites = siteinds("Fermion", N; conserve_qns = true)
    
    println("Making free fermion starting MPS")
    @time psi0_init = slater_determinant_to_mps(sites, Φ; eigval_cutoff=_eigval_cutoff, cutoff=_cutoff, maxdim=_maxlinkdim, maxblocksize=_maxblocksize)

    # Define SSH model Hamiltonian (free part)
    os = OpSum()
    for j in 1:N-1
        os += -t, "Cdag", j, "C", j + 1
        os += -t, "Cdag", j + 1, "C", j
    end
    os += -t, "Cdag", 1, "C", N
    os += -t, "Cdag", N, "C", 1

    # Define interacting term (-V n_j n_{j+1})
    os_interacting = OpSum()  
    for j in 1:N-1
        os_interacting += -V, "N", j, "N", j+1
    end
    os_interacting += -V, "N", N, "N", 1  # Periodic BC

    # Convert OpSum to MPO
    H = MPO(os + os_interacting, sites)

    # Perform DMRG calculations
    println("Running DMRG for ground state")
    energy_ground, psi_ground = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise)

    println("Running DMRG for first excited state")
    energy_excited1, psi_excited1 = dmrg(H, [psi_ground], psi0_init; nsweeps, maxdim, cutoff, noise, weight)

    println("Running DMRG for second excited state")
    energy_excited2, psi_excited2 = dmrg(H, [psi_ground, psi_excited1], psi0_init; nsweeps, maxdim, cutoff, noise, weight)

    # Compute Energy Gaps
    gap_1 = energy_excited1 - energy_ground
    gap_2 = energy_excited2 - energy_ground

    println("V = $V, Ground Energy: $energy_ground, First Excited Energy: $energy_excited1, Second Excited Energy: $energy_excited2")
    println("First Energy Gap: $gap_1, Second Energy Gap: $gap_2")
    
    return (gap_1, gap_2)
end

# Parameters
N = 64  # Number of sites   
t = 1.0  
V_values = collect(-4.6:0.4:6.0)  # Smooth range
nsweeps = 10
maxdim = [10, 20, 50, 100, 200, 400]
noise = 1E-10
cutoff = 1E-10 
weight = 5.0 

# Compute energy gaps
energy_gaps = [run_dmrg_ll(N, t, V, nsweeps, maxdim, cutoff, noise, weight) for V in V_values]

# Extract first and second energy gaps
gap_1_values = [gaps[1] for gaps in energy_gaps]
gap_2_values = [gaps[2] for gaps in energy_gaps]

# Plot results
plot(V_values, gap_1_values, marker=:o, linewidth=2, label="1st Energy Gap", color=:blue)
plot!(V_values, gap_2_values, marker=:s, linewidth=2, label="2nd Energy Gap", color=:red)
vline!([-2.5], linestyle=:dash, color=:gray, label="V = -2.5")

xlabel!("V")
ylabel!("Energy Gap")
title!("Energy Gaps vs Interaction V")

# Save and display the plot
savefig("ll_interacting_engaps_plot_test_osama.pdf")
display(plot!)

Making free fermion starting MPS
  6.658164 seconds (4.49 M allocations: 3.725 GiB, 41.08% gc time)
Running DMRG for ground state
After sweep 1 energy=-13.240418202952371  maxlinkdim=10 maxerr=1.82E-03 time=0.933
After sweep 2 energy=-13.246476381798232  maxlinkdim=20 maxerr=2.95E-08 time=0.650
After sweep 3 energy=-13.24650062144766  maxlinkdim=41 maxerr=9.83E-11 time=0.847
After sweep 4 energy=-13.255535514259439  maxlinkdim=55 maxerr=9.99E-11 time=1.003
After sweep 5 energy=-13.255540345201398  maxlinkdim=55 maxerr=9.99E-11 time=1.108
After sweep 6 energy=-13.255540352723031  maxlinkdim=55 maxerr=9.93E-11 time=1.133
After sweep 7 energy=-13.255540352822013  maxlinkdim=55 maxerr=9.98E-11 time=1.123
After sweep 8 energy=-13.25554035282413  maxlinkdim=55 maxerr=9.96E-11 time=1.157
After sweep 9 energy=-13.255540352823532  maxlinkdim=55 maxerr=9.97E-11 time=1.147
After sweep 10 energy=-13.255540352823319  maxlinkdim=55 maxerr=9.97E-11 time=1.155
Running DMRG for first excited state
Afte

plot! (generic function with 4 methods)